# Feature Extraction

A quick notebook to test and determine the fastest ways to extract features from the data.

In [1]:
from typing import *
from messages import *
from collections import Counter
import os

In [2]:
conversations: list[Conversation] = []

for user in os.listdir(FILE_PATH):
    conversation = Conversation(os.path.join(FILE_PATH, user))

    conversations.append(conversation)



In [3]:
direct = [conversation for conversation in conversations if len(conversation.participants) == 2 and conversation.title in conversation.participants]

In [5]:
def predictUser(directMessages: list[Conversation]) -> User:
    """Attempts to determine who the 'main' user is"""
    possibleUsers: list[User] = []

    for conv in directMessages:
        for name, user in conv.participants.items():
            if name != conv.title:
                possibleUsers.append(user)

    return max(Counter(possibleUsers))

predictUser(direct)

User("Moulik Budhiraja")